In [101]:
%pip install pandas -q
%pip install numpy -q

import numpy as np
import pandas as pd
import random
from collections import Counter


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Data Preparation

In [102]:
with open ("Dataset/Hurink_Data/vdata/abz5.fjs","r") as file:
    firstLine = file.readline()
    firstLineValues = [int(i) for i in firstLine.split()[0:2]]
    NUM_JOBS = firstLineValues[0] 
    NUM_MACHINES = firstLineValues[1]
    print(f"numJobs = {NUM_JOBS}, numMachines = {NUM_MACHINES}")
    
    data = []
    job_index = 0   
    for txts in file.readlines():
        job = []
        operations = txts.split()[1:]
        index = 0
        while index < len(operations):
            operation = []
            numberOfOperation = int(operations[index])
            index += 1
            endOperationIndex = numberOfOperation*2 + index
            while index < endOperationIndex:
                operation.append({"machineNumber": int(operations[index]), "processTime": int(operations[index+1])})
                index += 2
            job.append(operation)
        data.append(job)
    
# data = [job0, job1, .... ]
# job1 = , [op0, op1, op2, ....]
# op1 = [{macineNumber: , processTime: },{machineNumber: , processTime: }...]
print(f"number of first job's operation = {len(data[0])}")
print(f"number of machine that first job's operation can use = {len(data[0][0])}")

numJobs = 10, numMachines = 10
number of first job's operation = 10
number of machine that first job's operation can use = 5


## Hyper Parameter

In [103]:
POP_SIZE = 2
MUTATION_RATE = 0.3
CROSSOVER_RATE = 0.5
MAX_ITERATION = 400
NUMBER_OF_ELITISM_SURVIVORS = 2
NUMBER_OF_TOURNAMENT_COMPETITORS = 4

## Encoding

In [104]:
# Initial population = [[J0, M3], [J0, M4], [J2, M1], [J3, M2], [J0, M1], ...]
# meaning J1 first operation use Machine 3 -> J2 first operation use Machine 4  -> J1 second operation......

def generateInitailPopulation():
    # create all jobs with specific number of operations
    # because different Jobs could have different number of operations
    base_array = []
    for index in range(len(data)):   
        base_array.extend(np.tile(index, len(data[index])))
    
    population = []
    # random choose that jobs operation for
    for _ in range(POP_SIZE):
        # how many operations have done
        c = Counter()
        individual = []
        base_array = np.random.permutation(base_array)
        for job in base_array:
            machine = random.choice(data[job][c[job]])
            c[job]+=1
            individual.append([job, machine["machineNumber"]])
        population.append(individual)
    return population

## Fitness Evaluation

In [105]:
def evaluateFitnessScore(individual):
    
    # Job shows how many times of operation have been done
    # Job_end shows the time that last Ji ends
    # machine time shows the time that last job ends
    jobCounter = Counter()
    jobEndCounter = Counter()
    machineTimeCounter = Counter()
    
    for jobIteration in individual:
        jobNumber = jobIteration[0]
        machineNumber = jobIteration[1]
        
        # calculate the duration of that specific operation
        operationsDictList = data[jobNumber][jobCounter[jobNumber]]
        duration = [i["processTime"] for i in operationsDictList if i["machineNumber"] == machineNumber][0]
         
        # update the counter
        machineTimeCounter[machineNumber] = max(machineTimeCounter[machineNumber], jobEndCounter[jobNumber])
        machineTimeCounter[machineNumber] += duration
        jobEndCounter[jobNumber] = machineTimeCounter[machineNumber]
        jobCounter[jobNumber] += 1
        
    # calcualte makespan & tardiness
    makespan = jobEndCounter.most_common(1)[0][1]
    return makespan

## Selection

In [ ]:
# Tournament method
def selection(populatoinWithMakespan):
    population = populatoinWithMakespan["population"]
    makespan = populatoinWithMakespan["makespan"]
    selected = []
    for _ in range(2):
        # note: I decide to user random sample instead of permutation to increase diversity
        competitors = np.random.permutation(len(population))[:NUMBER_OF_TOURNAMENT_COMPETITORS]
        
        v = max(makespan[i] for i in competitors)
        for i in competitors:
            if makespan[i]==v:
                selected.append([population[i], makespan[i]])
                break
        
    return pd.DataFrame(selected,columns=["population", "makespan"])

##  Crossover & Mutation

In [ ]:
def crossover():
    return


def mutate():
    return

## Elitism Strategy

In [ ]:
def elitism_strategy(population):
    sorted_df = population.sort_values(by=["makespan"], ascending=[True])
    survivors_df = sorted_df.head(NUMBER_OF_ELITISM_SURVIVORS)
    return survivors_df

## Main Function

In [107]:
iteration = 0
populationList = []
lineGraphDataList = []

while(iteration <= MAX_ITERATION):
    if len(populationList) == 0 :
        populationList = generateInitailPopulation()
    else:
        populationList = sortedPopulation["population"]
        
    # scored the population
    scoredPopulationList = []
    for individual in populationList:
        makespan= evaluateFitnessScore(individual)
        scoredPopulationList.append([individual, makespan])
    
    # crossover & mutation 
    offspringList = []
    while len(offspring) <= POP_SIZE:
        selectedIndividual = selection(pd.DataFrame(scoredPopulationList,columns=["population", "makespan"]))
        if random.random < CROSSOVER_RATE:
            offsprings = crossover(selectedIndividual[0],selectedIndividual[1])
            for offspring in offsprings:
                makspan= evaluateFitnessScore(offspring)
                offspringList.append([offspring, makespan])
        else:
            offspring1 = mutate(selectedIndividual[0])
            makespan = evaluateFitnessScore(offspring1)
            offspringList.append([offspring1, makespan])

    
    print(f"interation: {iteration}")
    iteration+=1
    print(f"current population size: {len(offspringList)}")
    
    # sort by makespan (ascending)
    sortedPopulation = elitism_strategy(pd.DataFrame(offspringList))
    
    lineGraphDataList.append(sortedPopulation['makespan'].values[0])
    



In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(len(lineGraphDataList), lineGraphDataList, label='makespan')
plt.xlabel('Period')
plt.ylabel('Value')
plt.title('Trend of Makespan')
plt.legend()
plt.grid(True)
plt.show()